In [33]:
import tensorflow as tf
print("OneDNN enabled:", tf.config.optimizer.get_jit())

OneDNN enabled: 


In [34]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Force CPU usage
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Suppress INFO and WARNING messages

import tensorflow as tf
print("TensorFlow running on:", "GPU" if tf.config.list_physical_devices('GPU') else "CPU")

TensorFlow running on: CPU


In [35]:
print("OneDNN Enabled:", tf.config.optimizer.get_jit())  # Should print True or None

OneDNN Enabled: 


In [36]:
print("Available CPUs:", tf.config.list_physical_devices('CPU'))

Available CPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [37]:
import tensorflow as tf

# Adjust based on available CPU cores
tf.config.threading.set_inter_op_parallelism_threads(4)  # Controls operations running in parallel
tf.config.threading.set_intra_op_parallelism_threads(4)  # Controls parallelism within individual operations

print("Optimized TensorFlow settings for CPU")

Optimized TensorFlow settings for CPU


In [38]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from scipy.stats import chi2
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, ExtraTreeRegressor, ExtraTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor, ExtraTreesClassifier, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

from scipy.stats.mstats import winsorize
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings("ignore") # Suppress all warnings
pd.options.display.float_format = '{:.2f}'.format

In [39]:
# Initialize BigQuery and Storage clients
storage_client = storage.Client()
client = bigquery.Client()
storage_client = storage.Client()
print("Authentication successful!")


Authentication successful!


In [40]:
project_id = "big-data-engin"  # Replace with your GCP project ID
dataset_id = "Sample1"  # Replace with your BigQuery dataset ID
table_id = "airplane"    # Replace with your desired table name

full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Create an empty table with no schema
table = bigquery.Table(full_table_id)
table = client.create_table(table)
print(f"Table {full_table_id} created successfully.")


Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/big-data-engin/datasets/Sample1/tables?prettyPrint=false: Already Exists: Table big-data-engin:Sample1.airplane

In [41]:
gcs_uri = "gs://samplebucketvrs/Intelligence Analytics/og4/2019.csv"  # Replace with your GCS path

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skips the header row
    autodetect=True,  # Enables schema autodetection
    write_disposition="WRITE_APPEND",  # Append data if table already exists
)

load_job = client.load_table_from_uri(
    gcs_uri, full_table_id, job_config=job_config
)

load_job.result()  # Wait for the job to complete
print(f"Loaded CSV file into {full_table_id} successfully.")


Loaded CSV file into big-data-engin.Sample1.airplane successfully.


In [42]:
df = client.list_rows(table).to_dataframe()  # Download table contents into a DataFrame

# Display the DataFrame
df.head(5)

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,string_field_20
0,2019-08-25,AS,64,PSG,WRG,<NA>,NaN,NaN,<NA>,<NA>,...,<NA>,NaN,NaN,31.00,NaN,NaN,NaN,NaN,NaN,None
1,2019-01-16,AS,64,PSG,WRG,<NA>,NaN,NaN,<NA>,<NA>,...,<NA>,NaN,NaN,31.00,NaN,NaN,NaN,NaN,NaN,None
2,2019-12-09,AS,64,PSG,WRG,<NA>,NaN,NaN,<NA>,<NA>,...,<NA>,NaN,NaN,31.00,NaN,NaN,NaN,NaN,NaN,None
3,2019-10-27,AS,64,PSG,WRG,1455,-20.00,4.00,1459,1508,...,1512,-23.00,9.00,31.00,NaN,NaN,NaN,NaN,NaN,None
4,2019-05-02,AS,65,WRG,PSG,1028,-17.00,5.00,1033,1043,...,1047,-23.00,10.00,31.00,NaN,NaN,NaN,NaN,NaN,None


In [43]:
df.describe()
columns_to_drop = [col for col in df.columns if col.lower() == 'id']

if columns_to_drop:
    df = df.drop(columns=columns_to_drop)
# Convert the Date column to datetime
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

In [ ]:
def identify_column_type(df):
    column_types = []
    for col in df.columns:
        if pd.api.types.is_bool_dtype(df[col]):  # Check for Boolean columns first
            col_type = "Boolean"
        elif pd.api.types.is_numeric_dtype(df[col]):  # Check for numeric columns
            col_type = "Numerical"
        elif pd.api.types.is_categorical_dtype(df[col]) or df[col].dtype == 'object':  # Check for categorical columns
            col_type = "Categorical"
        elif pd.api.types.is_datetime64_any_dtype(df[col]):  # Check for datetime columns
            col_type = "Datetime"
        else:
            col_type = "Unknown"  # Any other type falls here
        column_types.append({'Column': col, 'Type': col_type})
    
    # Convert the list of dictionaries to a DataFrame
    column_types_df = pd.DataFrame(column_types)
    return column_types_df

# Identify column types
column_types_df = identify_column_type(df)
print(column_types_df)